In [ ]:
from splink import Linker, DuckDBAPI
import pandas as pd

db_api = DuckDBAPI()

In [ ]:
locs_df = pd.read_csv("production_locations_ds.csv")
locs_df = locs_df.drop(columns=[
    "created_at",
    "updated_at",
    "potential_match_score",
    "potential_match_os_id",
    "found_index"
])
sample_df = locs_df.sample(10000)
sample_df = sample_df[['os_id','name', 'address', 'country_code', 'geocoded_location_type', 'lat', 'lng']]
sample_df

In [ ]:

test_rec = locs_df.sample(1)[['os_id','name', 'address', 'country_code', 'geocoded_location_type', 'lat', 'lng']]
test_rec

In [ ]:
import json

with open("record_linkage_model_beta.json", "r") as file:
    record_linkage_model = json.load(file)

linker = Linker(
    sample_df,
    settings=record_linkage_model,
    db_api=db_api,
)

In [ ]:
df_predictions = linker.inference.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe()

In [ ]:
lk_df = df_predictions.as_pandas_dataframe().sort_values("match_weight", ascending=False)

In [ ]:
lk_df[lk_df["match_probability"] > 0.8].to_csv("lk_df.csv", index=False)

In [ ]:
df_inc = linker.inference.find_matches_to_new_records(
    sample_df.sample(10), blocking_rules=[]
).as_pandas_dataframe()
df_inc.sort_values("match_weight", ascending=False)